# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../result_data/weather.csv")

weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,severo-kurilsk,100,RU,1580372527,87,50.68,156.12,16.30,3.53
1,corn island,54,NI,1580372527,71,12.17,-83.04,79.39,10.60
2,kapaa,20,US,1580372462,83,22.08,-159.32,69.80,8.05
3,bahia blanca,0,AR,1580372525,37,-38.72,-62.27,57.47,16.69
4,ahome,0,MX,1580372528,94,25.92,-109.18,57.99,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [4]:
#Use the Lat and Lng as locations 
locations = weather_data[["Lat", "Lng"]].astype(float)

#Humidity as the weight.
humidity = weather_data["Humidity"].astype(float)


In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)



heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#Narrow down the cities to fit weather conditions.
narrowed_city_df = weather_data.loc[(weather_data["Max Temp"].astype(float) >= 60) &(weather_data["Max Temp"].astype(float) < 68) \
                               & (weather_data["Wind Speed"].astype(float) <= 8) & (weather_data["Cloudiness"].astype(float) == 0)]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
80,otane,0,NZ,1580372544,47,-39.88,176.63,66.99,5.01
150,abnub,0,EG,1580372560,42,27.27,31.15,62.80,7.36
208,coquimbo,0,CL,1580372573,87,-29.95,-71.34,60.80,2.24
312,santiago,0,CL,1580372382,67,-33.46,-70.65,60.80,3.36
323,iranshahr,0,IR,1580372373,22,27.20,60.68,66.20,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "keyword": "hotels",
    
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in narrowed_city_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    


    # convert to json
    name_address = name_address.json()
    

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        narrowed_city_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [15]:
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
80,otane,0,NZ,1580372544,47,-39.88,176.63,66.99,5.01,Craggy Range Vineyards
150,abnub,0,EG,1580372560,42,27.27,31.15,62.80,7.36,Florence Hotel Assiut
208,coquimbo,0,CL,1580372573,87,-29.95,-71.34,60.80,2.24,CasaMolle
312,santiago,0,CL,1580372382,67,-33.46,-70.65,60.80,3.36,Renaissance Santiago Hotel
323,iranshahr,0,IR,1580372373,22,27.20,60.68,66.20,4.70,Benyamin Pardis Inn
441,moron,0,AR,1580372622,87,-34.65,-58.62,60.80,3.36,Sheraton Buenos Aires Hotel & Convention Center
528,saint-louis,0,SN,1580372642,42,16.33,-15.00,66.20,2.24,La cour du fleuve
567,sardarshahr,0,IN,1580372651,33,28.43,74.48,66.16,3.74,LE PRINCE HAVELI (French Homestay)


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]


In [20]:
# Create a map using state centroid coordinates to set markers
marker_locations = narrowed_city_df[["Lat", "Lng"]].astype(float)


fig = gmaps.figure(center = [0,30] ,zoom_level = 2)

markers = gmaps.marker_layer(marker_locations,info_box_content= hotel_info)

fig.add_layer(heat_layer)
# Add marker layer ontop of heat map
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))